In [2]:
import pandas as pd
from pandas import Series, DataFrame
import pandas_profiling
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

plt.rc("font", size=14)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [3]:
data = sns.load_dataset('tips')
data.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [46]:
pandas_profiling.ProfileReport(data)

Number of variables,7
Number of observations,244
Total Missing (%),0.0%
Total size in memory,7.2 KiB
Average record size in memory,30.3 B
Numeric,3
Categorical,4
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [4]:
data  = pd.get_dummies(data=data,drop_first=True,columns=['sex','smoker','day','time'])
data.head()

,total_bill,tip,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
0,16.99,1.01,2,1,1,0,0,1,1
1,10.34,1.66,3,0,1,0,0,1,1
2,21.01,3.50,3,0,1,0,0,1,1
3,23.68,3.31,2,0,1,0,0,1,1
4,24.59,3.61,4,1,1,0,0,1,1


In [5]:
x_train,x_test,y_train,y_test = train_test_split(data[data.columns.difference(['total_bill'])],data.total_bill,test_size = 0.2,random_state = 123)

In [6]:
x_test.shape

(49, 8)

### building a KNN model on this data with total_bill being the dependent variablem

#### 1.6.4. Nearest Neighbor Algorithms

#### 1.6.4.1. Brute Force

Fast computation of nearest neighbors is an active area of research in machine learning. The most naive neighbor search implementation involves the brute-force computation of distances between all pairs of points in the dataset: for
samples in dimensions, this approach scales as . Efficient brute-force neighbors searches can be very competitive for small data samples. However, as the number of samples

grows, the brute-force approach quickly becomes infeasible. In the classes within sklearn.neighbors, brute-force neighbors searches are specified using the keyword algorithm = 'brute', and are computed using the routines available in sklearn.metrics.pairwise.


#### 1.6.4.2. K-D Tree

To address the computational inefficiencies of the brute-force approach, a variety of tree-based data structures have been invented. In general, these structures attempt to reduce the required number of distance calculations by efficiently encoding aggregate distance information for the sample. The basic idea is that if point
is very distant from point , and point is very close to point , then we know that points and are very distant, without having to explicitly calculate their distance. In this way, the computational cost of a nearest neighbors search can be reduced to or better. This is a significant improvement over brute-force for large

.

An early approach to taking advantage of this aggregate information was the KD tree data structure (short for K-dimensional tree), which generalizes two-dimensional Quad-trees and 3-dimensional Oct-trees to an arbitrary number of dimensions. The KD tree is a binary tree structure which recursively partitions the parameter space along the data axes, dividing it into nested orthotropic regions into which data points are filed. The construction of a KD tree is very fast: because partitioning is performed only along the data axes, no
-dimensional distances need to be computed. Once constructed, the nearest neighbor of a query point can be determined with only distance computations. Though the KD tree approach is very fast for low-dimensional () neighbors searches, it becomes inefficient as

grows very large: this is one manifestation of the so-called “curse of dimensionality”. In scikit-learn, KD tree neighbors searches are specified using the keyword algorithm = 'kd_tree', and are computed using the class KDTree.

References:

    “Multidimensional binary search trees used for associative searching”, Bentley, J.L., Communications of the ACM (1975)

#### 1.6.4.3. Ball Tree

To address the inefficiencies of KD Trees in higher dimensions, the ball tree data structure was developed. Where KD trees partition data along Cartesian axes, ball trees partition data in a series of nesting hyper-spheres. This makes tree construction more costly than that of the KD tree, but results in a data structure which can be very efficient on highly structured data, even in very high dimensions.

A ball tree recursively divides the data into nodes defined by a centroid
and radius , such that each point in the node lies within the hyper-sphere defined by and

. The number of candidate points for a neighbor search is reduced through use of the triangle inequality:

With this setup, a single distance calculation between a test point and the centroid is sufficient to determine a lower and upper bound on the distance to all points within the node. Because of the spherical geometry of the ball tree nodes, it can out-perform a KD-tree in high dimensions, though the actual performance is highly dependent on the structure of the training data. In scikit-learn, ball-tree-based neighbors searches are specified using the keyword algorithm = 'ball_tree', and are computed using the class sklearn.neighbors.BallTree. Alternatively, the user can work with the BallTree class directly.

References:

    “Five balltree construction algorithms”, Omohundro, S.M., International Computer Science Institute Technical Report (1989)

In [6]:
from sklearn.neighbors import KNeighborsRegressor

`KNeighborsRegressor(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=None, **kwargs)`

In [7]:
gscv_knn = GridSearchCV(KNeighborsRegressor(),{'n_neighbors' : [3,4,5,6,7],
                                               'algorithm' : ['ball_tree', 'kd_tree', 'brute'],'p':[1,2],
                                               'leaf_size':[10,20,20,40]},scoring='r2',cv =5)

In [8]:
gscv_knn.fit(x_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [3, 4, 5, 6, 7], 'algorithm': ['ball_tree', 'kd_tree', 'brute'], 'p': [1, 2], 'leaf_size': [10, 20, 20, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [9]:
gscv_knn.best_params_

{'algorithm': 'kd_tree', 'leaf_size': 10, 'n_neighbors': 6, 'p': 2}

In [10]:
gscv_knn.best_score_

0.3627325567327492

# --------------------------------

In [11]:
knn = KNeighborsRegressor(n_neighbors=7,algorithm='kd_tree',p=2,leaf_size=10)
knn

KNeighborsRegressor(algorithm='kd_tree', leaf_size=10, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=7, p=2,
          weights='uniform')

In [12]:
knn.fit(x_train,y_train)

KNeighborsRegressor(algorithm='kd_tree', leaf_size=10, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=7, p=2,
          weights='uniform')

In [13]:
knn_pred = knn.predict(x_test)
knn_pred

array([21.65      , 19.48571429, 25.20571429, 17.35571429, 18.7       ,
       30.64571429, 16.25428571, 18.11142857, 20.80714286, 15.85857143,
       13.39285714, 11.19      , 25.34714286, 13.92714286, 19.38428571,
       20.51428571, 12.31      , 17.71857143, 24.62142857, 17.82      ,
       19.89285714, 14.93142857, 13.36142857, 20.82571429, 12.31      ,
       22.45571429, 29.03      , 30.73      , 14.02428571, 15.41142857,
       25.91428571, 12.31      , 18.50714286, 22.45571429, 21.59857143,
       11.74285714, 12.31      , 27.86428571, 23.72142857, 14.02      ,
       22.12      , 18.56714286, 16.72285714, 13.96857143, 29.15857143,
       18.39428571, 15.41142857, 17.66428571, 11.74285714])

In [25]:
knn.get_params()

{'algorithm': 'kd_tree',
 'leaf_size': 10,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 7,
 'p': 2,
 'weights': 'uniform'}

In [33]:
knn.n_neighbors

7

In [34]:
knn.weights

'uniform'

In [18]:
final = pd.DataFrame({'predict':knn.predict(x_test),'actual':y_test})
final.head(2)

,predict,actual
112,21.650000,38.07
19,19.485714,20.65


#### MAE error & R2 error

In [51]:
print('Mean of actual:',final.actual.mean())
print('std of actual:',final.actual.std())

mae = metrics.mean_absolute_error(final['actual'],final['predict'])
'MAE: ',mae

Mean of actual: 20.495306122448973
std of actual: 8.898960169134186


('MAE: ', 4.397784256559767)

#### custom error function based on std_devation

In [41]:
std_error = np.abs(final.actual - final.predict)/final.actual.std() 
print(std_error.mean(),std_error.median())

0.4941908012818585 0.38126444869650966


#### R2 error

In [31]:
metrics.r2_score(final['actual'],final['predict'])

0.5216716890605733